In [ ]:
import xml.etree.ElementTree as et
import glob
import re
import pandas as pd
from collections import OrderedDict
import sqlalchemy as sa
    
res = []

def compute_xml_data(lst,elem, func, level=0):
    func(lst,elem,level)
    for child in elem.getchildren():
        compute_xml_data(lst,child, func, level+1)

def gather_level(lst,elem,level):
    lst.append(('-'*level+elem.tag, elem.text))

xml_files = glob.glob('./*.xml')

for xml_file in xml_files:
    lst = []
    root = et.parse(xml_file)
    compute_xml_data(lst,root.getroot(), gather_level)
    res.append(lst)

def get_table_names(xml_data_list): 
    table_names = []
    for t in xml_data_list:
        if t[0]:
            if re.match(r'(-{http://webservices.irb.com/}\w)', t[0], re.IGNORECASE):
                name = t[0][30:].strip()
                table_names.append(name)
    return table_names
                
tables = get_table_names(res[0])

for xml_file in xml_files:
    lst = []
    root = et.parse(xml_file)
    compute_xml_data(lst,root.getroot(), gather_level)

# build all the dataframe for the xml
def build_xml_dfs(xmls):
    matchID_tag = '--{http://webservices.irb.com/}MatchID'
    match_info_tag = '-{http://webservices.irb.com/}MatchInformation'
    officials_tag = '-{http://webservices.irb.com/}Officials'
    official_tag = '--{http://webservices.irb.com/}Official'
    team1TeamSheet_tag = '-{http://webservices.irb.com/}Team1TeamSheet'
    team2TeamSheet_tag = '-{http://webservices.irb.com/}Team2TeamSheet'
    player_tag = '--{http://webservices.irb.com/}Player'
    team1Scores_tag = '-{http://webservices.irb.com/}Team1Scores'
    team2Scores_tag = '-{http://webservices.irb.com/}Team2Scores'
    score_tag = '--{http://webservices.irb.com/}Score'
    team1Substitutions_tag = '-{http://webservices.irb.com/}Team1Substitutions'
    team2Substitutions_tag = '-{http://webservices.irb.com/}Team2Substitutions'
    substitution_tag = '--{http://webservices.irb.com/}Substitution'
    team1Cards_tag = '-{http://webservices.irb.com/}Team1Cards'
    team2Cards_tag = '-{http://webservices.irb.com/}Team2Cards'
    card_tag = '--{http://webservices.irb.com/}Card'
    
    MatchID_idx = [xml[0] for xml in xmls].index(matchID_tag)
    MatchID = xmls[MatchID_idx][1]
    
    match_info_list = []
    officials_list = []
    team1TeamSheet_list = []
    team2TeamSheet_list = []
    team1Scores_list = []
    team2Scores_list = []
    team1Substitutions_list = []
    team2Substitutions_list = []
    team1Cards_list = []
    team2Cards_list = []

    for t in xmls:
        if t[0] == match_info_tag: 
            match_info_start_idx = [xml[0] for xml in xmls].index(match_info_tag)
            officials_start_idx = [xml[0] for xml in xmls].index(officials_tag)
            team1TeamSheet_start_idx = [xml[0] for xml in xmls].index(team1TeamSheet_tag)
            team2TeamSheet_start_idx = [xml[0] for xml in xmls].index(team2TeamSheet_tag)
            team1Scores_start_idx = [xml[0] for xml in xmls].index(team1Scores_tag)
            team2Scores_start_idx = [xml[0] for xml in xmls].index(team2Scores_tag)
            team1Substitutions_start_idx = [xml[0] for xml in xmls].index(team1Substitutions_tag)
            team2Substitutions_start_idx = [xml[0] for xml in xmls].index(team2Substitutions_tag)
            team1Cards_start_idx = [xml[0] for xml in xmls].index(team1Cards_tag)
            team2Cards_start_idx = [xml[0] for xml in xmls].index(team2Cards_tag)
            
            #print(team1Cards_start_idx)
            #print(team2Cards_start_idx)

            for m in xmls[match_info_start_idx+1:officials_start_idx]:
                if m[0] is not officials_tag:
                    match_info_list.append((m[0][31:].strip(),m[1]))
                else:
                    continue
            for o in xmls[officials_start_idx+1:team1TeamSheet_start_idx]:
                if o[0] == official_tag:
                    sub_list = []
                    sub_list.append(('MatchID',MatchID))
                    officials_list.append(sub_list)
                elif o[0] is not team1TeamSheet_tag:
                    sub_list.append((o[0][32:].strip(),o[1]))
                else:
                    continue
            for t1 in xmls[team1TeamSheet_start_idx+1:team2TeamSheet_start_idx]: #25/132
                if t1[0] == player_tag:
                    sub_list = []
                    sub_list.append(('MatchID',MatchID))
                    team1TeamSheet_list.append(sub_list)
                elif t1[0] is not team2TeamSheet_tag:
                    sub_list.append((t1[0][32:].strip(),t1[1]))
                else:
                    continue
            for t2 in xmls[team2TeamSheet_start_idx+1:team1Scores_start_idx]:
                if t2[0] == player_tag:
                    sub_list = []
                    sub_list.append(('MatchID',MatchID))
                    team2TeamSheet_list.append(sub_list)
                elif t2[0] is not team1Scores_tag:
                    sub_list.append((t2[0][32:].strip(),t2[1]))
                else:
                    continue
            for t3 in xmls[team1Scores_start_idx+1:team2Scores_start_idx]:
                if t3[0] == score_tag:
                    sub_list = []
                    sub_list.append(('MatchID',MatchID))
                    team1Scores_list.append(sub_list)
                elif t3[0] is not team2Scores_tag:
                    sub_list.append((t3[0][32:].strip(),t3[1]))
                else:
                    continue
            for t4 in xmls[team2Scores_start_idx+1:team1Substitutions_start_idx]:
                if t4[0] == score_tag:
                    sub_list = []
                    sub_list.append(('MatchID',MatchID))
                    team2Scores_list.append(sub_list)
                elif t4[0] is not team1Substitutions_tag:
                    sub_list.append((t4[0][32:].strip(),t4[1]))
                else:
                    continue
            if (team2Substitutions_start_idx - team1Substitutions_start_idx)>1:
                for t5 in xmls[team1Substitutions_start_idx+1:team2Substitutions_start_idx]:
                    if t5[0] == substitution_tag:
                        sub_list = []
                        sub_list.append(('MatchID',MatchID))
                        team1Substitutions_list.append(sub_list)
                    elif t5[0] is not team2Substitutions_tag:
                        sub_list.append((t5[0][32:].strip(),t5[1]))
                    else:
                        continue
            else:
                team1Substitutions_list
            if (team1Cards_start_idx - team2Substitutions_start_idx)>1:
                for t6 in xmls[team2Substitutions_start_idx+1:team1Cards_start_idx]:
                    if t6[0] == substitution_tag:
                        sub_list = []
                        sub_list.append(('MatchID',MatchID))
                        team2Substitutions_list.append(sub_list)
                    elif t6[0] is not team1Cards_tag:
                        sub_list.append((t6[0][32:].strip(),t6[1]))
                    else:
                        continue
            else:
                team2Substitutions_list.append([('MatchID',MatchID)])
            if (team2Cards_start_idx - team1Cards_start_idx)>1:
                for t7 in xmls[team1Cards_start_idx+1:team2Cards_start_idx]:
                    if t7[0] == card_tag:
                        sub_list = []
                        sub_list.append(('MatchID',MatchID))
                        team1Cards_list.append(sub_list)
                    elif t7[0] is not team2Cards_tag:
                        sub_list.append((t7[0][32:].strip(),t7[1]))
                    else:
                        continue
            else: 
                team1Cards_list.append([('MatchID',MatchID), ('PersonID',None), ('PersonName',None),('Type',None),('Time',None),('Half',None)])
            if not (team2Cards_start_idx+1 == len(xmls)):
                for t8 in xmls[team2Cards_start_idx+1:]:
                    if t8[0] == card_tag:
                        sub_list = []
                        sub_list.append(('MatchID',MatchID))
                        team2Cards_list.append(sub_list)
                    elif t8[0]:
                        sub_list.append((t8[0][32:].strip(),t8[1]))
                    else:
                        break
            else:
                team2Cards_list.append([('MatchID',MatchID), ('PersonID',None), ('PersonName',None),('Type',None),('Time',None),('Half',None)])
                break

    records_dict = {'match_info': match_info_list, 'officials': officials_list, 'team1TeamSheet': team1TeamSheet_list, 'team2TeamSheet':team2TeamSheet_list, 'team1Scores': team1Scores_list, 'team2Scores': team2Scores_list, 'team1Substitutions': team1Substitutions_list, 'team2Substitutions': team2Substitutions_list, 'team1Cards': team1Cards_list, 'team2Cards': team2Cards_list}
    
    match_info_df = pd.DataFrame.from_records(match_info_list).transpose()
    match_info_df = match_info_df.rename(columns=match_info_df.iloc[0]).drop(match_info_df.index[0])
    
    officials_df = [pd.DataFrame.from_records(lst).transpose() for lst in officials_list]
    officials_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in officials_df]
    
    team1TeamSheet_df = [pd.DataFrame.from_records(lst).transpose() for lst in team1TeamSheet_list]
    team1TeamSheet_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in team1TeamSheet_df]
    
    team2TeamSheet_df = [pd.DataFrame.from_records(lst).transpose() for lst in team2TeamSheet_list]
    team2TeamSheet_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in team2TeamSheet_df]
    
    team1Scores_df = [pd.DataFrame.from_records(lst).transpose() for lst in team1Scores_list]
    team1Scores_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in team1Scores_df]
    
    team2Scores_df = [pd.DataFrame.from_records(lst).transpose() for lst in team2Scores_list]
    team2Scores_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in team2Scores_df]

    team1Substitutions_df = [pd.DataFrame.from_records(lst).transpose() for lst in team1Substitutions_list]
    team1Substitutions_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in team1Substitutions_df]
    
    team2Substitutions_df = [pd.DataFrame.from_records(lst).transpose() for lst in team2Substitutions_list]
    team2Substitutions_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in team2Substitutions_df]

    team1Cards_df = [pd.DataFrame.from_records(lst).transpose() for lst in team1Cards_list]
    team1Cards_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in team1Cards_df]
    
    team2Cards_df = [pd.DataFrame.from_records(lst).transpose() for lst in team2Cards_list]
    team2Cards_df = [df.rename(columns=df.iloc[0]).drop(df.index[0]) for df in team2Cards_df]

    match_info_dfs_merged = match_info_df
    if officials_df: 
        officials_dfs_merged = pd.concat(officials_df)
    else:
        officials_dfs_merged = pd.DataFrame()
    if team1TeamSheet_df:
        team1TeamSheet_dfs_merged = pd.concat(team1TeamSheet_df)
    else:
        team1TeamSheet_dfs_merged = pd.DataFrame()
    if team2TeamSheet_df:
        team2TeamSheet_dfs_merged = pd.concat(team2TeamSheet_df)
    else:
        team2TeamSheet_dfs_merged = pd.DataFrame()
    if team1Scores_df:
        team1Scores_dfs_merged = pd.concat(team1Scores_df)
    else:
        team1Scores_dfs_merged = pd.DataFrame()
    if team2Scores_df:
        team2Scores_dfs_merged = pd.concat(team2Scores_df)
    else:
        team2Scores_dfs_merged = pd.DataFrame()
    if team1Substitutions_df:
        team1Substitutions_dfs_merged = pd.concat(team1Substitutions_df)
    else:
        team1Substitutions_dfs_merged = pd.DataFrame()
    if team2Substitutions_df:
        team2Substitutions_dfs_merged = pd.concat(team2Substitutions_df)
    else:
        team2Substitutions_dfs_merged = pd.DataFrame()
    if team1Cards_df:
        team1Cards_dfs_merged = pd.concat(team1Cards_df)
    else:
        team1Cards_dfs_merged = pd.DataFrame()
    if team2Cards_df:
        team2Cards_dfs_merged = pd.concat(team2Cards_df)
    else:
        team2Cards_dfs_merged = pd.DataFrame()

    return {'match_info':match_info_dfs_merged,
            'officials':officials_dfs_merged,
            'team1TeamSheet':team1TeamSheet_dfs_merged,
            'team2TeamSheet':team2TeamSheet_dfs_merged,
            'team1Scores':team1Scores_dfs_merged,
            'team2Scores':team2Scores_dfs_merged,
            'team1Substitutions':team1Substitutions_dfs_merged,
            'team2Substitutions':team2Substitutions_dfs_merged,
            'team1Cards':team1Cards_dfs_merged,
            'team2Cards':team2Cards_dfs_merged}

all_xml_dfs = []
match_info_dfs_list = []
officials_dfs_list = []
team1TeamSheet_dfs_list = []
team2TeamSheet_dfs_list = []
team1Scores_dfs_list = []
team2Scores_dfs_list = []
team1Substitutions_dfs_list = []
team2Substitutions_dfs_list = []
team1Cards_dfs_list = []
team2Cards_dfs_list = []

for xml_data in res:
    all_xml_dfs.append(build_xml_dfs(xml_data))

for df_dict in all_xml_dfs:
    for k,v in df_dict.items():
        if k == 'match_info':
            match_info_dfs_list.append(v)
        if k == 'officials':
            officials_dfs_list.append(v)
        if k == 'team1TeamSheet':
            team1TeamSheet_dfs_list.append(v)
        if k == 'team2TeamSheet':
            team2TeamSheet_dfs_list.append(v)
        if k == 'team1Scores':
            team1Scores_dfs_list.append(v)
        if k == 'team2Scores':
            team2Scores_dfs_list.append(v)
        if k == 'team1Substitutions':
            team1Substitutions_dfs_list.append(v)
        if k == 'team2Substitutions':
            team2Substitutions_dfs_list.append(v)
        if k == 'team1Cards':
            team1Cards_dfs_list.append(v)
        if k == 'team2Cards':
            team2Cards_dfs_list.append(v)

if match_info_dfs_list:
    match_info_dfs_merged_all = pd.concat(match_info_dfs_list)
else:
    match_info_dfs_merged_all = pd.DataFrame()
if officials_dfs_list:
    officials_dfs_merged_all = pd.concat(officials_dfs_list)
else:
    officials_dfs_merged_all = pd.DataFrame()
if team1TeamSheet_dfs_list:
    team1TeamSheet_dfs_merged_all = pd.concat(team1TeamSheet_dfs_list)
else:
    team1TeamSheet_dfs_merged_all = pd.DataFrame()
if team2TeamSheet_dfs_list:
    team2TeamSheet_dfs_merged_all = pd.concat(team2TeamSheet_dfs_list)
else:
    team2TeamSheet_dfs_merged_all = pd.DataFrame()
if team1Scores_dfs_list:
    team1Scores_dfs_merged_all = pd.concat(team1Scores_dfs_list)
else:
    team1Scores_dfs_merged_all = pd.DataFrame()
if team2Scores_dfs_list:
    team2Scores_dfs_merged_all = pd.concat(team2Scores_dfs_list)
else:
    team2Scores_dfs_merged_all = pd.DataFrame()
if team1Substitutions_dfs_list:
    team1Substitutions_dfs_merged_all = pd.concat(team1Substitutions_dfs_list)
else:
    team1Substitutions_dfs_merged_all = pd.DataFrame()
if team2Substitutions_dfs_list:
    team2Substitutions_dfs_merged_all = pd.concat(team2Substitutions_dfs_list)
else:
    team2Substitutions_dfs_merged_all = pd.DataFrame()  
if team1Cards_dfs_list:
    team1Cards_dfs_merged_all = pd.concat(team1Cards_dfs_list)
else:
    team1Cards_dfs_merged_all = pd.DataFrame()
if team2Cards_dfs_list:
    team2Cards_dfs_merged_all = pd.concat(team2Cards_dfs_list)
else:
    team2Cards_dfs_merged_all = pd.DataFrame()  

# load dataframe into Postgres
def write_data_to_sql(df, table_name):        
    try:
        engine = sa.create_engine(connection_string, echo=True);
        # add a table id?
        # df["TableId"] = tableId;
        if not df.empty:
            df.to_sql(tablename, engine, if_exists='append', index=False)       
    except Exception as e:
        print(e)

match_info_dfs_merged_all.to_csv('../_3_data_clean/match_matchinfo.csv',index=False)
officials_dfs_merged_all.to_csv('../_3_data_clean/match_officials.csv',index=False)
team1TeamSheet_dfs_merged_all.to_csv('../_3_data_clean/match_team1teamsheet.csv',index=False)
team2TeamSheet_dfs_merged_all.to_csv('../_3_data_clean/match_team2teamsheet.csv',index=False)
team1Scores_dfs_merged_all.to_csv('../_3_data_clean/match_team1scores.csv',index=False)
team2Scores_dfs_merged_all.to_csv('../_3_data_clean/match_team2scores.csv',index=False)
team1Substitutions_dfs_merged_all.to_csv('../_3_data_clean/match_team1substitutions.csv',index=False)
team2Substitutions_dfs_merged_all.to_csv('../_3_data_clean/match_team2substitutions.csv',index=False)
team1Cards_dfs_merged_all.to_csv('../_3_data_clean/match_team1cards.csv',index=False)
team2Cards_dfs_merged_all.to_csv('../_3_data_clean/match_team2cards.csv',index=False)
print('Completed!')


In [ ]:
#run below script to find out the xml format, only run if format is due to change. 
import xml.etree.ElementTree as et
import glob
import re
import pandas as pd
from collections import OrderedDict
import sqlalchemy as sa
    
res = []

def compute_xml_data(elem, func, level=0):
    func(elem,level)
    for child in elem.getchildren():
        compute_xml_data(child, func, level+1)

def gather_level(elem,level):
    print('-'*level+elem.tag, elem.text)
#     res.append(('-'*level+elem.tag, elem.text))

xml_files = glob.glob('./*.xml')
xml_file = xml_files[1]
root = et.parse(xml_file) 
xml_data = compute_xml_data(root.getroot(), gather_level)

def get_table_names(xml_data_list): 
    table_names = []
    for t in xml_data_list:
        if t[0]:
            if re.match(r'(-{http://webservices.irb.com/}\w)', t[0], re.IGNORECASE):
                name = t[0][30:].strip()
                table_names.append(name)
    return table_names

# print(res)

print(get_table_names(res))

# compute_xml_data(root.getroot(), gather_level)
